In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
colors = [(1,1,1), colors[1]]
C1_cmap = mpl.colors.LinearSegmentedColormap.from_list("C1", colors, N=20)

In [4]:
n = 50
Λ = model_problem_spectrum(n,.6,1e3)
A = np.diag(Λ)
b = np.ones(n)
b /= np.linalg.norm(b)

k_max = 30
Q,(α,β) = lanczos_reorth(A,b,k_max,reorth=0)
Q_ro,(α_ro,β_ro) = lanczos_reorth(A,b,k_max,reorth=k_max)

T = np.diag(α) + np.diag(β[:k_max-1],1) + np.diag(β[:k_max-1],-1)
T_ro = np.diag(α_ro) + np.diag(β_ro[:k_max-1],1) + np.diag(β_ro[:k_max-1],-1)

T_err = np.abs(T-T_ro)
Q_err = np.abs(Q[:,:k_max] - Q_ro[:,:k_max])
QQI_err = np.abs(Q[:,:k_max].T@Q[:,:k_max] - np.eye(k_max))

In [5]:
np.linalg.eigvalsh(T)[-10:]

array([ 203.57273469,  345.96074429,  345.96081633,  345.96081633,
        588.16734694,  588.16734694,  588.16734694, 1000.        ,
       1000.        , 1000.        ])

In [6]:
np.linalg.eigvalsh(T_ro)[-10:]

array([   9.21846392,   15.03987587,   24.97051977,   41.90207739,
         70.75548082,  119.90138776,  203.57273469,  345.96081633,
        588.16734694, 1000.        ])

In [7]:
Λ[-10:]

array([   9.21846392,   15.03987587,   24.97051977,   41.90207739,
         70.75548082,  119.90138776,  203.57273469,  345.96081633,
        588.16734694, 1000.        ])

In [8]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

axs = []
axs.append(fig.add_axes([left, bottom, width2, height]))
axs.append(fig.add_axes([left+hoffset2, bottom, width2, height]))
axs.append(fig.add_axes([left, bottom, width, .02]))

for ax in axs:
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])

im = axs[0].imshow(T_err,norm=mpl.colors.LogNorm(vmin=1e-16,vmax=1),cmap=C1_cmap)
axs[1].imshow(QQI_err,norm=mpl.colors.LogNorm(vmin=1e-16,vmax=1),cmap=C1_cmap)

fig.colorbar(im,cax=axs[2],orientation='horizontal')

plt.savefig('imgs/lanczos_unstable.pdf')
plt.close()

In [9]:
e0 = np.arange(k_max)==0
m = get_chebyshev_moments(T,e0,k_max,min(Λ),max(Λ))
m_ro = get_chebyshev_moments(T_ro,e0,k_max,min(Λ),max(Λ))

In [10]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])
ax.plot(np.abs(m-m_ro),**line_styles['l1'],ls='None')

ax.set_xlabel('degree $n$')
ax.set_ylabel('error')
ax.set_yscale('log')
ax.set_ylim(1e-16,1)

plt.savefig('imgs/moment_err.pdf')
plt.close()

In [11]:
α_e,β_e,_ = extend_t(A,α,β,Q,verbose=0)

T_e = np.diag(α_e) + np.diag(β_e[:-1],1) + np.diag(β_e[:-1],-1)
T_e = T_e.astype(np.double)

Λ_e,V_e = np.linalg.eigh(T_e)

In [12]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])

for λ,ω in zip(Λ,b**2):
    plt.plot([λ,λ],[0,ω],**line_styles_nomarker['l1'])
ax.plot(Λ,b**2,**line_styles['l1'],ls='None')

for λ,ω in zip(Λ_e,V_e[0]**2):
    plt.plot([λ,λ],[0,ω],**line_styles_nomarker['l2'])
ax.plot(Λ_e,V_e[0]**2,**line_styles['l2'],ls='None')


ax.grid(True,linestyle=':',linewidth=.5)

ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax.set_yscale('symlog',linthresh=1e-20)
ax.set_yticks([1e-20,1e-15,1e-10,1e-5,1e0])

ax.set_xscale('log')
ax.set_xlabel('eigenvalue position: $x$')
ax.set_ylabel('density')

plt.savefig('imgs/finite_precision_greenbaum.pdf')
plt.close()